In [8]:
import os
import json
import pickle
import collections as C
import itertools as I

from common.constants import CORE_OPTIONS

In [15]:
with open('/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/output/sft_ar_v2/Goedel-Prover-V2-8B.Numina-Lean.problem_generator/problem_generation.20250822-005458.pkl', 'rb') as f:
    data = pickle.load(f)

In [51]:
results_all = C.defaultdict(list)

In [52]:
for ((ptype, source), idx), v in data.items():
    results_all[(ptype, source)].append(v)

In [53]:
header = ("""
import Mathlib
import Aesop

""" + '\n'.join('set_option ' + t.replace('=', ' ') for t in CORE_OPTIONS)).strip()
print(header)

import Mathlib
import Aesop

set_option maxHeartbeats 0
set_option maxRecDepth 100000
set_option tactic.hygienic false
set_option pp.fullNames true
set_option pp.funBinderTypes true
set_option pp.piBinderTypes true


In [54]:
for (ptype, source), ds in results_all.items():
    with open(f'/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/data/MiniF2F/Goedel-Prover-V2-8B.Numina-Lean.problem_generator/{ptype}.{source}.lean', 'w') as f:
        f.write(header + '\n\n')
        for result in ds:
            if result.formal_statement == '':
                continue
            meta = json.loads(result.metainfo)
            if 'original_trajectory' in meta.keys():
                meta.pop('original_trajectory')
                meta['is_reassembled'] = True
            f.write('-- ' + json.dumps(meta) + '\n')
            
            steps = result.steps
            submission_name = steps[-1].step[len('submit_answer '):]
            submission_fvar = [v for v in result.trajectory[-1][0] if v.name == submission_name]
            assert len(submission_fvar) == 1, f'submission_name={submission_name}, new_context={[v.name for v in steps[-1].new_contexts]}'
            submission_fvar = submission_fvar[0]
            
            # Construct statement and proof
            problem_hypotheses = []
            for s in steps:
                if s.is_introducing:
                    step_code = s.step
                    assert step_code.startswith('have ') and step_code.endswith(' := sorry')
                    problem_hypotheses.append('(' + step_code[len('have '):-len(' := sorry')] + ')')
            
            formal_statement = 'example\n' + '\n'.join(problem_hypotheses) + '\n: ' + submission_fvar.t + '\n:= by\n'
            f.write(formal_statement + result.formal_solution_draft + '\n\n')

In [42]:
len(ds)

1

In [12]:
d

(('Abstract Algebra', 'unknown'), 14)